<a href="https://colab.research.google.com/github/nemanja899/Time-Seris-Analysis/blob/master/Generate_Time_Series_Syntetic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Plot Utilities

Utility funkcija za plotovanje run chart-a vremenskih serija. Kod koristi vizulelizuje numpy niz u graf koristeci Pyplot bliblioteku. Na X osi se nalaze koraci vremenske serije. Brojevi na x osi predstavljau cisto cele brojeve, mogu recimo da budu godine, dani , sati itd... Na Y-osi ce se nalaziti odgovarajuce vrednosti po vremenu.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    """
    Vizuelizuje vremensku seriju

    Args:
      time (array of int) - sadrzi korake serije
      series (array of int) - sadrzi vrednosti serije u koraku
      format (string) - stil linije za plotovanje
      start (int) - pocetna vrednost grafa
      end (int) - poslednja vrednost grafa
      label (list of strings)- tagovi za legendu
    """

    # Velicina dimenzija
    plt.figure(figsize=(10, 6))

    # Plotuje podatke
    plt.plot(time[start:end], series[start:end], format)

    
    plt.xlabel("Time")

   
    plt.ylabel("Value")

    if label:
      plt.legend(fontsize=14, labels=label)

    # Postavlja mrezu na graf
    plt.grid(True)

    # prikazuje nacrtani grafikon
    plt.show()

## Trend

Trend opisuje globalnu tendenciju da vrednosti idu gore ili dole kako vreme otice. Datim vremenskim periodom mozemo videti da li graf prati gornji ili donji trend ili samo ravna linija.

Sledeca funkcija crta trend kao pravu liniju. 
Y=slope*X+constant

In [ ]:
def trend(time, slope=0,constant=0):
    """
   Generise sinteticku seriju koja pravi pravu liniju

    Args:
      time (array of int) - vremenske korake
      slope (float) - koliki ce biti nagib

    Returns:
      series (array of float) - vrednosti koje prate pravu liniju
    """

    
    series = slope * time+constant

    return series

In [ ]:
# Generisemo koraka. Recimo jedan korak je jedan dan (365 dana)
time = np.arange(365)

# Definisemo ugao nagiba,, moze da bude promenljivo!!
slope = 0.3
constant=0

# Generisemo funkciju nagiba
series = trend(time, slope,constant)

# Prikaz rezultata
plot_series(time, series, label=[f'slope={slope}'])

## Sezonalnost

Jos jedan atribut vremenske serije je sezonalnost. Ovo je ponavljajuci patern koji se ponovo pojavljuje nakon nekog vremenskog intervala.

Moze da se koristi funkcija ispod za generisanje nekog oscilovanja vremenske serije.

In [ ]:
def seasonal_pattern(season_time):
    """
    Pravi jedan patern
    Args:
      season_time (array of float) - sadrzi merenja tokom vemena

    Returns:
      data_pattern (array of float) -  vraca novu generisanu seriju sa sezonalnoscu
    """

    # Generisemo neki patern sa nekim arbitrarnim vrednostima ovo moze da se menja
    data_pattern = np.where(season_time < 0.62,
                    np.cos(season_time * 2 * np.e),
                    1 / np.exp(np.e * season_time))
    
    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    """
   Ponavlja iste paterne

    Args:
      time (array of int) - vreme koraka
      period (int) - broj koraka nakong cega patern serije ponavlja
      amplitude (int) - najveca izmerena vrednost u priodu sluzi da se podignu vrednosti serije
      phase (int) - broj koraka da se pomere izmerene vrednosti

    Returns:
      data_pattern (array of float) - vraca seriju sklaliranu amplitudom
    """
    
    # pravi vrednosti serije izmedju 0 i 1 , prvo racuna ostatak, a zatim racuna procenat ostatka tj procenat u toj sezoni dokle se stiglo
    season_time = ((time + phase) % period) / period

   
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern

In [ ]:
# Generise korake recimo 5 godina plus jedan dan
time = np.arange(5 * 365 + 1)

# parametri
period = 365
amplitude = 45

# generise sezonalnost
series = seasonality(time, period=period, amplitude=amplitude)

plot_series(time, series)

Vremesnka serija moze da sadrzi i trend i sezonu. Na primer temperatura koja se godisnje ponavlja, a zbog uticaja staklene baste ima trend nagore.

Primer dole pokazuje kombinaciju trenda i sezone

In [ ]:
#Parametri
slope = 0.01
period = 365
amplitude = 30

#Generisanje serije , naime mozemo samo da saberemo vrednosti dve funkcije trenda i sezionalnosti
series = trend(time, slope) + seasonality(time, period=period, amplitude=amplitude)

plot_series(time, series)

## Beli sum

U praksi , rekte vremenske serije imace gladak signal. Uglavnom imaju neki sum preko signala. 

In [ ]:
def noise(time, noise_level=1, seed=None):
    """Generise sum normalne distribucije

    Args:
      time (array of int) - koraci u vremenu
      noise_level (float) - faktor koji se mnozi za vrednostima iz normalne raspodele
      seed (int) - generator slucajnih brojeva

    Returns:
      noise (array of float) -  beli sum

    """

    # inicijalizacija generatora slucajnih brojeva 
    rnd = np.random.RandomState(seed)

    # slucajni brojevi sa normalnom raspodelom koji se mnoze sa faktorom
    noise = rnd.randn(len(time)) * noise_level
    
    return noise


In [ ]:
# faktor buke
noise_level = 6

# generise beli sum
noise_signal = noise(time, noise_level=noise_level, seed=39)

plot_series(time, noise_signal)

Sada da dodamo sve i trend i sezonalnost i beli sum.

In [ ]:
# sabiramo dve funkcije, sabira elemenat po elemenat , moraju biti iste duzine
series += noise_signal

plot_series(time, series)

Kao sto mozemo da vidimo i dalje ima blagi trend nagore ali ima vise varijacija izmedju koraka zbog dodatog suma.

## Autokorelacija

Vremenska serija moze imati i autokorelaciju. To znaci da mera jednog koraka moze da zavisi od merenja iz vise prethodnih. Tako da sadasnje vreme je neka funkcija nekih proslih koraka. 

In [ ]:
def autocorrelation_2_order(time, amplitude, seed=None):
    """
    Generise autokorelacione podatke

    Args:
      time (array of int) - vremenske korake
      amplitude (float) - faktor skaliranja
      seed (int) - generator slucajnih brojeva

    Returns:
      ar (array of float) - autokorelacioni podaci
    """

    
    rnd = np.random.RandomState(seed)
    
    #Povecavamo velicinu niza jer prvih 60 brojeva nije korelisano, tim brojevima dodelicemo neku konstantu da nam pomogne za racunanje
    #u sledecoj autokorelaciji , zatim cemo ih odbaciti
    # generisanje niza sa normalnom raspodelom
    ar = rnd.randn(len(time) + 60)
    
    # Postavljanje prvih 50 brojeva kao konstanta moze da se stavi bilo koji broj
    ar[:60] = 50
    
    # Posto je ovo autokorelacija reda 2 , biramo dva faktora phi. Napomena faktori moraju biti izmedju -1 i 1 ne ukljucujuci ih
    # jer ako dobiju suprutnu vrenost nece konvergirati vec ce teziti vremenom beskonacnosti
    phi1 = 0.5
    phi2 = -0.2

    # Autokorelisati  element od 60 pa nadalje sa nekim proizvoljnim merama iz proslosti recimo (t-45) i (t-24)
    # ,gde je t trenutni korak koji zelimo izracunati na osnovu prethodnih
    # ovde je formula x(t)=phi1*x(t-45)+phi2*(t-24)+constanta+et
    # et je vrednost suma koja je generisana rnd.randn(len(time) + 60) komandom i koja se sabira sa odgovarajucim vrednostima, ovde je konstanta jednaka nuli
    for step in range(60, len(time) + 60):
        ar[step] += phi1 * ar[step - 45]
        ar[step] += phi2 * ar[step - 24]
    
    #vracamo autokorelacionu seriju od 60 koraka nadalje i mnozimo je faktorom visine
    ar = ar[60:] * amplitude

    return ar

In [ ]:

series = autocorrelation_2_order(time, amplitude=16, seed=39)

# Prikaz prvih 150 vrednosti da bi se autokorelacija bolje prikazala
plot_series(time[:150], series[:150])

Autokorelacija prvog reda x(t)=phi*x(t-to)+et+constanta

In [ ]:
def autocorrelation_1_order(time, amplitude, seed=None):
    """
    Autokorelacija prvog reda

    Args:
      time (array of int) - koraci serije
      amplitude (float) - faktor skaliranja
      seed (int) - generator slucajnih brojeva

    Returns:
      ar (array of float) - autokorelacioni podaci
    """

    
    rnd = np.random.RandomState(seed)

    # Sada naredna vrednost zavisi direktno od prethodne nekom funkcijom
    ar = rnd.randn(len(time) + 1)

    # Define scaling factor
    phi = 0.8

    # Auto korelirati 1 element pa nadalje
    # (t-1)
    for step in range(1, len(time) + 1):
        ar[step] += phi * ar[step - 1]
    
    ar = ar[1:] * amplitude
    
    return ar

In [ ]:
# Serija sa autokorelacijom reda 1
series = autocorrelation_1_order(time, amplitude=10, seed=42)

# Plot the results
plot_series(time[:200], series[:200])

Jos jedan tip korelacije na koje se moze naici su serije koje se urusavaju preditkivno nakon nasumicnih impulsa.

In [ ]:
def impulses(time, num_impulses, amplitude=1, seed=None):
    """
   Generise slucajne impulse

    Args:
      time (array of int) - vremenski koraci
      num_impulses (int) - koliko impulsa da se generise
      amplitude (float) - skalirani faktor
      seed (int) - generator slucajnoh brojeva

    Returns:
      series (array of float) - niz koji sadrzi impulse
    """

    
    rnd = np.random.RandomState(seed)

    # Generise slucajne cele brojeve tj indekse u kom vremenu ce se naci impulsi
    impulse_indices = rnd.randint(len(time), size=num_impulses)

    #niz nula sa velicinom koliko ima i vremenskih koraka
    series = np.zeros(len(time))

    # dodeljivanje vrednosti impulsa u impulsnim indeksima koje smo prethodno napravili. 0+(normalna_rasp*faktor)= vrednost impulsa
    for index in impulse_indices:
        series[index] += rnd.rand() * amplitude

    return series    

In [ ]:
# Generisanje slucajnih impulsa
impulses_signal = impulses(time, num_impulses=15, seed=42)


plot_series(time, impulses_signal)

Sada kada imamo vrednosti impulsa mozemo da napravimo funkciju urusavanja(decay)

In [ ]:
def autocorrelation_impulses(source, phis):
    """
    Generise autokorelacionu seriju od impulsa

    Args:
      source (array of float) - vremenska serija sa impulsima
      phis (dict) - dictionary koji sadrzi lag i faktor urusavanja

    Returns:
      ar (array of float) - autokorelacioni podaci
    """

    # Kopiramo izvor sa vrednostima impulsa
    ar = source.copy()

    # Pravimo novu seriju koja ima faktor urusavanja
    for step, value in enumerate(source):
        for lag, phi in phis.items():
            if step - lag > 0:
              ar[step] += phi * ar[step - lag]

    return ar

In [ ]:
# Koristi impulse iz prethodnog koraka i pravi seriju sa urusavanjem
# Autokorelacija reda 1, lag je jedan sto znaci da je naredna vrednost 0.97* prethodna vrednost
series = autocorrelation_impulses(impulses_signal, {1: 0.97})

plot_series(time, series)

Jos jedan primer autokorelacije reda 2 sa lagom jedan i 45 `t-1` i `t-45`:

In [ ]:
# Use the impulses from the previous section and generate autocorrelated data
series = autocorrelation_impulses(impulses_signal, {1: 0.65, 45: 0.3})

# Plot the results
plot_series(time, series)

Autokorelacioni podaci mogu da imaju i trend

In [ ]:
# Autokorelacioni podaci sa trendom
series = autocorrelation_1_order(time, 10, seed=42) + trend(time, 2)

plot_series(time[:200], series[:200])

Slicno i sezonalnost moze da se doda na podatke

In [ ]:
# Sezonalnost sa trendom i autokorelacijom i belim sumom
series = autocorrelation_1_order(time, 10, seed=42) + seasonality(time, period=60, amplitude=150) + trend(time, 0.5)

# Plot the results
plot_series(time[:200], series[:200])


## Ne-stacionarne vremenske serije

Moguce je i da vremenske seriju prekinu predvidjeni pater. Veliki dogadjaji mogu da budu uzrok tome ili neko sezonsko ponasanje.

In [ ]:
# Slicna serija kao gore sa sezonama, autokorelacijom reda 1, trendom i belim sumom
series = autocorrelation_1_order(time, 10, seed=42) + seasonality(time, period=60, amplitude=150) + trend(time, 0.5)

# generisanje nove serije sa nekim negativnim trendom. Broj na kraju je arbitraran da bi se nadovezao lepo na seriju
# eksperimentisanjem se doslo do njega
series2 = autocorrelation_1_order(time, 5, seed=42) + seasonality(time, period=30, amplitude=5) + trend(time, -1.8) + 500

#Uzimanje prvih dvesta podataka od druge serije i dodeljivanje prvoj od 200 pa nadalje
series[200:] = series2[200:]

plot_series(time[:300], series[:300])